# 13. Redesneuronales recurrentes
[**Python Deep Learning** Introducción práctica con Keras y TensorFlow 2. Jordi Torres. Editorial Marcombo ISBN: 9788426728289 ](https://www.marcombo.com/python-deep-learning-9788426728289/)

### Importar TensorFlow 2.0  y otras librerias

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras

import numpy as np
import os
import time

TensorFlow 2.x selected.


### Descarga del conjunto de datos

In [0]:

from google.colab import files
# se debe cargar el fichero “Libro-Deep-Learning-introduccion-practica-con-Keras-1a-parte.txt”
#files.upload()

#path_to_fileDL ='/content/Libro-Deep-Learning-introduccion-practica-con-Keras-1a-parte.txt'

#path_to_fileDL = tf.keras.utils.get_file('Shakespear.txt', 'https://cs.stanford.edu/people/karpathy/char-rnn/shakespear.txt')


In [4]:
#PARA BORRAR EN VERSIÓN DEFINITIVA
path_to_fileDL = tf.keras.utils.get_file('DL-Introduccion-practica-con-Keras-1a.txt', 'https://raw.githubusercontent.com/jorditorresBCN/Deep-Learning-Introduccion-practica-con-Keras/master/DeepLearning-Introduccion-practica-con-Keras-PRIMERA-PARTE.txt')

204800/203286 [==============================] - 0s 0us/step


In [5]:


text = open(path_to_fileDL, 'rb').read().decode(encoding='utf-8')
print('Longitud del texto:        {} carácteres'.format(len(text)))

vocab = sorted(set(text))

print ('El texto está compuesto de estos {} carácteres:'.format(len(vocab)))
print (vocab)

Longitud del texto:        203251 carácteres
El texto está compuesto de estos 92 carácteres:
['\n', '\r', ' ', '!', '"', '#', '%', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\xad', 'ÿ', 'Š', '‡', '…']


In [6]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

for char,_ in zip(char2idx, range(len(vocab))):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))

  '\n':   0,
  '\r':   1,
  ' ' :   2,
  '!' :   3,
  '"' :   4,
  '#' :   5,
  '%' :   6,
  "'" :   7,
  '(' :   8,
  ')' :   9,
  '*' :  10,
  '+' :  11,
  ',' :  12,
  '-' :  13,
  '.' :  14,
  '/' :  15,
  '0' :  16,
  '1' :  17,
  '2' :  18,
  '3' :  19,
  '4' :  20,
  '5' :  21,
  '6' :  22,
  '7' :  23,
  '8' :  24,
  '9' :  25,
  ':' :  26,
  ';' :  27,
  '<' :  28,
  '=' :  29,
  '>' :  30,
  '?' :  31,
  '@' :  32,
  'A' :  33,
  'B' :  34,
  'C' :  35,
  'D' :  36,
  'E' :  37,
  'F' :  38,
  'G' :  39,
  'H' :  40,
  'I' :  41,
  'J' :  42,
  'K' :  43,
  'L' :  44,
  'M' :  45,
  'N' :  46,
  'O' :  47,
  'P' :  48,
  'Q' :  49,
  'R' :  50,
  'S' :  51,
  'T' :  52,
  'U' :  53,
  'V' :  54,
  'W' :  55,
  'X' :  56,
  'Y' :  57,
  '[' :  58,
  ']' :  59,
  '_' :  60,
  'a' :  61,
  'b' :  62,
  'c' :  63,
  'd' :  64,
  'e' :  65,
  'f' :  66,
  'g' :  67,
  'h' :  68,
  'i' :  69,
  'j' :  70,
  'k' :  71,
  'l' :  72,
  'm' :  73,
  'n' :  74,
  'o' :  75,
  'p' :  76,

In [0]:
text_as_int = np.array([char2idx[c] for c in text])

In [8]:
print ('texto: {}'.format(repr(text[:50])))
print ('{}'.format(repr(text_as_int[:50])))

texto: 'Prologo\r\nEn 1953, Isaac Asimov publico Segunda Fun'
array([48, 78, 75, 72, 75, 67, 75,  1,  0, 37, 74,  2, 17, 25, 21, 19, 12,
        2, 41, 79, 61, 61, 63,  2, 33, 79, 69, 73, 75, 82,  2, 76, 81, 62,
       72, 69, 63, 75,  2, 51, 65, 67, 81, 74, 64, 61,  2, 38, 81, 74])


### Preparar los datos para entrenar la RNN


In [0]:
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

seq_length = 100
 
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)



In [10]:
for item in sequences.take(10):
  print(repr(''.join(idx2char[item.numpy()])))

'Prologo\r\nEn 1953, Isaac Asimov publico Segunda Fundacion, el tercer libro de la saga de la Fundacion '
'(o el decimotercero segun otras fuentes, este es un tema de debate). En Segunda Fundacion aparece por'
' primera vez Arkady Darell, uno de los principales personajes de la parte final de la saga. En su pri'
'mera escena, Arkady, que tiene 14 anos, esta haciendo sus tareas escolares. En concreto, una redaccio'
'n que lleva por titulo ?El Futuro del Plan Sheldon?. Para hacer la redaccion, Arkady esta utilizando '
'un ?transcriptor?,un dispositivo que convierte su voz en palabras escritas. Este tipo de dispositivo,'
' que para Isaac Asimov era ciencia ficcion en 1953, lo tenemos al alcance de la mano en la mayoria de'
' nuestros smartphones, y el Deep Learning es uno de los responsables de que ya tengamos este tipo de '
'aplicaciones, siendo la tecnologia otro de ellos.En la actualidad disponemos de GPUs (Graphics Proces'
'sor Units), que solo cuestan alrededor de 100 euros, que esta

In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)


In [12]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Prologo\r\nEn 1953, Isaac Asimov publico Segunda Fundacion, el tercer libro de la saga de la Fundacion'
Target data: 'rologo\r\nEn 1953, Isaac Asimov publico Segunda Fundacion, el tercer libro de la saga de la Fundacion '


In [13]:
print (dataset)

<MapDataset shapes: ((100,), (100,)), types: (tf.int64, tf.int64)>


In [14]:
BATCH_SIZE = 64

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

print (dataset)

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>


### Construcción del modelo

In [0]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

In [0]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = Sequential()
  model.add(Embedding(input_dim=vocab_size, 
                      output_dim=embedding_dim, 
                      batch_input_shape=[batch_size, None]))
  model.add(LSTM(rnn_units,
                 return_sequences=True,
                 stateful=True,
                 recurrent_initializer='glorot_uniform'))
  model.add(Dense(vocab_size))
  return model

In [0]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           23552     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 92)            94300     
Total params: 5,364,828
Trainable params: 5,364,828
Non-trainable params: 0
_________________________________________________________________


In [0]:
for input_example_batch, target_example_batch in dataset.take(1):
  print("Input:", input_example_batch.shape, "# (batch_size, sequence_length)")
  print("Target:", target_example_batch.shape, "# (batch_size, sequence_length)")


Input: (64, 100) # (batch_size, sequence_length)
Target: (64, 100) # (batch_size, sequence_length)


In [0]:
for input_example_batch, target_example_batch in dataset.take(1):  
    example_batch_predictions = model(input_example_batch)
    print("Prediction: ", example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

Prediction:  (64, 100, 91) # (batch_size, sequence_length, vocab_size)


In [0]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices_characters = tf.squeeze(sampled_indices,axis=-1).numpy()

In [0]:
sampled_indices_characters

array([14, 15, 14, 59, 53, 79, 89, 37, 55, 49, 61, 78, 25, 56, 78, 14, 48,
       80, 22,  2, 63, 13, 21, 84, 84, 72, 26, 15, 19, 62, 22, 78, 42, 80,
       56,  7,  2, 75, 72, 25, 38, 70, 13, 43, 14, 69, 53, 56,  4, 20, 14,
        2, 75, 68,  9, 35, 54, 40, 20, 21,  5, 81, 44, 59, 73, 81,  2, 65,
       72, 81,  2, 10, 77, 60, 54, 12, 70, 79, 64, 43, 26, 31, 41,  6, 28,
        7,  0, 21, 49, 49, 43,  6,  3, 79, 17, 16, 78, 38, 14, 50])

Entrenar el modelo

In [0]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [0]:
model.compile(optimizer='adam', loss=loss)

Configurar el *checkpoints*

In [0]:
 # directorio
checkpoint_dir = './training_checkpoints'
# nombre fichero
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

*Training*

In [0]:
EPOCHS=50
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Train for 31 steps
Epoch 1/50
31/31 [==============================] - 4s 116ms/step - loss: 3.2211
Epoch 2/50
31/31 [==============================] - 2s 75ms/step - loss: 2.7647
Epoch 3/50
31/31 [==============================] - 2s 75ms/step - loss: 2.4131
Epoch 4/50
31/31 [==============================] - 2s 73ms/step - loss: 2.2176
Epoch 5/50
31/31 [==============================] - 2s 74ms/step - loss: 2.1037
Epoch 6/50
31/31 [==============================] - 2s 73ms/step - loss: 1.9737
Epoch 7/50
31/31 [==============================] - 2s 73ms/step - loss: 1.8459
Epoch 8/50
31/31 [==============================] - 2s 73ms/step - loss: 1.7187
Epoch 9/50
31/31 [==============================] - 2s 73ms/step - loss: 1.6065
Epoch 10/50
31/31 [==============================] - 2s 73ms/step - loss: 1.4985
Epoch 11/50
31/31 [==============================] - 2s 73ms/step - loss: 1.4070
Epoch 12/50
31/31 [==============================] - 2s 74ms/step - loss: 1.3238
Epoch 13/50
31/31

### Generación de texto

In [0]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_50'

In [0]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [0]:
def generate_text(model, start_string):

  num_generate = 500
  input_eval = [char2idx[s] for s in start_string]

  input_eval = tf.expand_dims(input_eval, 0)
  text_generated = []


  temperature = 0.5

  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      
      predictions = tf.squeeze(predictions, 0)

      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()


      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [0]:
print(generate_text(model, start_string=u"a"))

al de que se deben con el filtro y el punado de 25 neuronas (5?5) de la capa de entrada.
Ahora bien, lo particular y muy importante en el resultado del aprendizaje.  Estos de tamano que comentaremos mas adelante, en el proceso de entrenamiento aparecera el concepto de overfitting (al que tambien referenciaremos por su tratado con una sola generado es una excelencia de cada uno de sus pixeles a esa clase. Para explicar los casos en que el modelo en cada paso la nueva generacion de supercomputacion
